In [4]:


# Imports
from PIL import Image
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from skimage import io
# from sys.path.append("/Users/jordbeek/Documents/Artificial Intelligence/Year 3/BP/CNN/Resnet CNN.ipynb") import ResNet
%run "Resnet CNN.py"

from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches

# HERE BELOW: ALTER FOR PERSONAL DATASET
class DeforestationDataset(Dataset):
    def __init__(self, csv_file, root_dir, gis_dir, transform=None):
        self.annotations = pd.read_csv(csv_file, header=None, names=['file_name', 'loss'])
        self.root_dir = root_dir
        self.transform = transform
        self.all_images = os.listdir(self.root_dir)
        self.gis_images = os.listdir(self.gis_dir)
        

    def __len__(self):
#         return len(self.annotations) #23865
       return len(os.listdir(self.root_dir)) 
    
    def __getitem__(self, index):
        name = self.all_images[index]
        image = io.imread(os.path.join(self.root_dir, name))[..., :3]
        gis_image = io.imread(os.path.join(self.gis_dir, name))[..., :3]
    
        
        
        # name_in_DSLabel = self.annotations.iloc[index, 0]
        # Loss Label_9_3.tif
        # Last - Edit_97_31.tif
#         name = 'Last - Edit' + name_in_DSLabel.split('l')[-1]
#         print(name_in_DSLabel)
#         print(name)
#         img_path = os.path.join(self.root_dir, name)
#         image = io.imread(img_path)
#         image = Image.open(img_path)
#         y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image_A = self.transform(image)
            image_B = self.transform(gis_image)
            A_B = torch.cat([A,B], 0) # Check dimensions
            print(A_B)
            
        name_in_csv = 'Loss Label' + name[11:]        
        y_label = torch.tensor(float(self.annotations['loss'][annotations['file_name'] == name_in_csv]), dtype=torch.float)

        return (A_B, y_label)


# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channel = 3
num_classes = 2
learning_rate = 1e-3
batch_size = 4
num_epochs = 10

# Load Data
labels_csv = open('/Users/jordbeek/Documents/Artificial Intelligence/Year 3/BP/Datasets/Global Deforestation/Bolivia East/Labels.csv')
deforestation_train_dir = ('/Users/jordbeek/Documents/Artificial Intelligence/Year 3/BP/Datasets/Global Deforestation/Bolivia East/TrainSet/')
'deforestation_test_dir = (\'/Users/jordbeek/Documents/Artificial Intelligence/Year 3/BP/Datasets/Global Deforestation/Bolivia East/TestSet/\')'
gis_train_dir = ('path') # Will put path in once dataset is created
'gis_test_dir = (\'path\')' # ^
train_dataset = DeforestationDataset(
    csv_file=labels_csv, 
    root_dir=deforestation_train_dir,
    gis_dir = gis_train_dir
    transform=transforms.ToTensor(),
)


# train_set, test_set = torch.utils.data.random_split(dataset, [5, 5]) # split dataset -- not necessary
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)


# Model
model = ResNet50(img_channels=3)
model.to(device)

# Loss and optimizer
# criterion = nn.CrossEntropyLoss() # Regression: mean squared error loss
criterion = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Gradient Descent?

# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        print(scores.shape)
        print(targets.shape)
        loss = criterion(scores, targets)
        print(scores[0])
        print(targets[0])
        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

torch.Size([2, 1])
tensor([[0.0205],
        [0.2173]], grad_fn=<AddmmBackward0>)


TypeError: __init__() missing 1 required positional argument: 'gis_dir'